# Collect list of files with CCTP content and EBP reference

In [5]:
import pandas as pd
import numpy as np
import re
import os # This module is great for path manipulation

from sklearn.model_selection import cross_val_score
from skrub import TableReport
from skrub import tabular_learner

#file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\listfile_FAMARO2.txt"
#file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO.txt"
file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO\result_scan_AO_v4.txt"

pd.set_option('display.width', 500)

Extraction du contenu du log de script scannant les repertoires onedrive des CSPS
- num commande EBP => construction df_EBP
- filename de CCTP, CCAP, RC,  => construction df_consult

In [ ]:
# init variables
EBP = False
CCTP = False
REJET = False

nb_valid_EBP = 0
nb_EBP_file = 0

nb_valid_CCTP = 0
nb_CCTP_file = 0
consult_non_valide = 0

nb_valid_rejet = 0
nb_rejet_file = 0
rejet_non_valide = 0

df_EBP = pd.DataFrame()
df_consult = pd.DataFrame()
df_rejet = pd.DataFrame()

# pour chaque ligne du fichier
with open(file_script_result, 'r', encoding='utf-8') as file:
    print(f"Contenu de '{file_script_result}':")
    for line_number, line in enumerate(file, 1):
        cur_line = line.strip()

        # reinit variables
        numero_commande = ""
        ebp_status = "unknown"

        if "=== Liste ref EBP ===" in line:
            print("Processing des EBP")
            EBP = True
            CCTP = False
        elif "=== Liste fichiers CONSULT ===" in cur_line:
            print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des CCTP")
            EBP = False
            CCTP = True
        elif "=== Liste fichiers REJET ===" in cur_line:
            line_num_consult = line_number-line_num_ebp
            print("total CONSULT file:", nb_CCTP_file, " Valid CONSULT:", nb_valid_CCTP, " non valide:",consult_non_valide," nb lines read:",line_num_consult)
            print("Processing des REJETs")
            EBP = False
            CCTP = False
            REJET = True
        else:
            # Extraction info du path

            ##### CSPS Name extraction
            parts = line.split("\\ATAE\\") 
            #print(len(parts), line)
            if len(parts) > 1:
                remaining_path = parts[1]
                first_backslash_index = remaining_path.find('\\')
                #print(remaining_path, first_backslash_index)
                if first_backslash_index != -1:
                    extracted_name = remaining_path[:first_backslash_index]
                    #print(f"Nom extrait : {extracted_name}")
                else:
                    extracted_name = "Not found (no backslash after ATAE)"
                    continue
            else:
                extracted_name = "Not found (ATAE not in path)"
                continue

            ##### Full File Path extraction
            directory_path = os.path.dirname(cur_line)
            parts_for_folder = cur_line.split(extracted_name, 1) # Split by the extracted name, once
            if len(parts_for_folder) > 1:
                path_after_name = parts_for_folder[1] # This starts with the first backslash after the name
                path_before_filename = os.path.dirname(cur_line)
                name_end_pos = cur_line.find(extracted_name) + len(extracted_name)
                
                escaped_extracted_name = re.escape(extracted_name)   
                idx_name = cur_line.find(extracted_name)
                
                idx_filename_start = cur_line.rfind(os.sep) # Last separator

                if idx_name != -1 and idx_filename_start != -1:
                    start_index_segment = idx_name + len(extracted_name) + 1
                    end_index_segment = idx_filename_start + 1 # Include the trailing backslash

                    if start_index_segment < end_index_segment:
                        directory_name = cur_line[start_index_segment:end_index_segment]
                        
                        # EBP status en fonction du path
                        if "00-DEVIS" in directory_name or "00- DEVIS" in directory_name or "02-DEVIS A" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Offre en cours de decision"
                        elif "01-CHANTIERS" in directory_name or "01- CHANTIERS" in directory_name or "01 - CHANTIERS" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Chantiers"
                        elif "02-AO DEVIS A FAIRE" in directory_name:
                            ebp_status = "Devis en cours"
                        elif "ARCHIVE" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Archive"
                    else:
                        directory_name = "Segment invalide"
                else:
                    directory_name = "Segment invalide"

                ##### Filename extraction
                file_name = os.path.basename(cur_line)
                #print(f"Nom du fichier : {file_name}")

            if EBP and "invalide" not in directory_name :
                if "de ebp " in line.lower():
                    nb_EBP_file += 1
                    match_numero = re.search(r"DE EBP (\d+)", cur_line, re.IGNORECASE)
                    numero_commande = match_numero.group(1) if match_numero else "Non trouvé"
                
                    #before_commande_ebp = cur_line.lower().split("de ebp", )[0]
                    before_commande_ebp = cur_line.split("DE EBP")[0]
                    #print(before_commande_ebp)
                    #match_details = re.search(r"\\([^\\]+)-([^\\]+)-([^\\]+)$", before_commande_ebp)
                    match_details = re.search(r"\\([^\\]+)-([^\\]+)-([^\\]*?)\\", after_echec)
                    
                    if match_details:
                        ville_code_dep = match_details.group(1)
                        entreprise = match_details.group(2)
                        mission_name = match_details.group(3)
                        
                        #print(f"Commande:{numero_commande} Ville:{ville_code_dep} Entreprise:{entreprise} Mission:{mission_name}")
                        nb_valid_EBP += 1

                        # Create a new row as a dictionary
                        new_row_EBP = {
                            'SPS Name': extracted_name.strip(),
                            'Directory': directory_name.strip(),
                            'ID EBP': numero_commande.strip(),
                            'Ville': ville_code_dep.strip(),
                            'Entreprise': entreprise.strip(),
                            'Mission': mission_name.strip(),
                            'statut': ebp_status,
                        }
                        df_EBP = pd.concat([df_EBP, pd.DataFrame([new_row_EBP])], ignore_index=True)
                    else:
                        #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
                        pass

            if CCTP and "invalide" not in directory_name:
                if "consult" in cur_line.lower():
                    #print(cur_line)
                    nb_CCTP_file += 1
                    match = re.search(r"\\consult", line, re.IGNORECASE)
                    if match:
                        before_consult = line[:match.start()]
                        match_details = re.search(r"\\([^\\]+)-([^\\]+)-(.*)\\([^\\]+)$", before_consult)

                        if match_details:
                            ville_code_dep = match_details.group(1)
                            entreprise = match_details.group(2)
                            mission_name = match_details.group(3)
                            #print(f"Ville:{ville_code_dep} Entreprise:{entreprise} Mission:{mission_name}")
                            nb_valid_CCTP += 1

                            new_row_consult = {
                            'SPS Name': extracted_name.strip(),
                            'Directory': directory_name.strip(),
                            'ID EBP': numero_commande.strip(),
                            'Ville': ville_code_dep.strip(),
                            'Entreprise': entreprise.strip(),
                            'Mission': mission_name.strip(),
                            'statut': ebp_status,
                            'filename': file_name.strip(),
                            }
                            df_consult = pd.concat([df_consult, pd.DataFrame([new_row_consult])], ignore_index=True)
                        else:
                            consult_non_valide += 1
                            #print("Détails Consultation (ville, entreprise, rénovation) : Non trouvés", before_consult)
                            pass

            if REJET and "invalide" not in directory_name:
                if "notification" in cur_line.lower() or "refus" in cur_line.lower() or "regret" in cur_line.lower() or "analyse" in cur_line.lower():
                    #print(cur_line)
                    ebp_status = "Perdu"
                    nb_rejet_file += 1
                    match = re.search(r"ECHEC 20", line, re.IGNORECASE)
                    if match:
                        after_echec = line[match.start():]
                        #match_details = re.search(r"\\([^\\]+)-([^\\]+)-(.*)\\([^\\]+)$", after_echec)
                        match_details = re.search(r"\\([^\\]+)-([^\\]+)-([^\\]*?)\\", after_echec)


                        if match_details:
                            ville_code_dep = match_details.group(1)
                            entreprise = match_details.group(2)
                            mission_name = match_details.group(3)
                            nb_valid_rejet += 1

                            new_row_rejet = {
                            'SPS Name': extracted_name.strip(),
                            'Directory': directory_name.strip(),
                            'ID EBP': numero_commande, # ""
                            'Ville': ville_code_dep.strip(),
                            'Entreprise': entreprise.strip(),
                            'Mission': mission_name.strip(),
                            'statut': ebp_status,    # Perdu
                            'filename': file_name.strip(),
                            'filepath': line.strip()
                            }
                            df_rejet = pd.concat([df_rejet, pd.DataFrame([new_row_rejet])], ignore_index=True)
                        else:
                            rejet_non_valide += 1
                            #print("Détails REJET (ville, entreprise, rénovation) : Non trouvés", after_echec)
                            pass

line_rejet = line_number - line_num_consult
print("total REJET file:", nb_rejet_file, " Valid REJET:", nb_valid_rejet, " non valide:",rejet_non_valide," nb lines read:",line_number - line_num_consult)


Contenu de 'C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO\result_scan_AO_v4.txt':
Processing des EBP
total EBP file: 3343  Valid EBP: 3043 nb line read: 3353
Processing des CCTP
total CONSULT file: 5137  Valid CONSULT: 1951  non valide: 2007  nb lines read: 143153
Processing des REJETs
total REJET file: 244  Valid REJET: 174  non valide: 70  nb lines read: 3597


In [8]:
TableReport(df_rejet)

Processing column   9 / 9


,,,,,,,,,


In [3]:
print("Total fichiers scannés:", line_rejet+line_num_ebp+line_num_consult)
print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP)
print("total Consult file:", nb_CCTP_file, " Valid Consult:", nb_valid_CCTP)
print("total Rejet file:", nb_rejet_file, " Valid Rejet:", nb_valid_rejet)
print("")


Total fichiers scannés: 150103
total EBP file: 3343  Valid EBP: 3043
total Consult file: 5137  Valid Consult: 1951
total Rejet file: 244  Valid Rejet: 174



In [4]:
df_EBP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3043 entries, 0 to 3042
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    3043 non-null   object
 1   Directory   3043 non-null   object
 2   ID EBP      3043 non-null   object
 3   Ville       3043 non-null   object
 4   Entreprise  3043 non-null   object
 5   Mission     3043 non-null   object
 6   statut      3043 non-null   object
dtypes: object(7)
memory usage: 166.5+ KB


In [5]:
df_consult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    1951 non-null   object
 1   Directory   1951 non-null   object
 2   ID EBP      1951 non-null   object
 3   Ville       1951 non-null   object
 4   Entreprise  1951 non-null   object
 5   Mission     1951 non-null   object
 6   statut      1951 non-null   object
 7   filename    1951 non-null   object
dtypes: object(8)
memory usage: 122.1+ KB


In [3]:
df_rejet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    174 non-null    object
 1   Directory   174 non-null    object
 2   ID EBP      174 non-null    object
 3   Ville       174 non-null    object
 4   Entreprise  174 non-null    object
 5   Mission     174 non-null    object
 6   statut      174 non-null    object
 7   filename    174 non-null    object
dtypes: object(8)
memory usage: 11.0+ KB


In [6]:
TableReport(df_EBP)


Processing column   7 / 7


,,,,,,,


In [7]:

filtered_ebp = df_EBP[
    (df_EBP['SPS Name'] == "Cyrille CHARTIER - CYRILLE") &
    (df_EBP['statut'] == "Chantiers") &
    (df_EBP['Entreprise'] == "MAIRIE")  
].head(10)

print(filtered_ebp)

                      SPS Name                                          Directory  ID EBP            Ville Entreprise                                        Mission     statut
1   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\CHAUMES EN RETZ - MAIR...  250492  CHAUMES EN RETZ     MAIRIE                         ancien bureau de poste  Chantiers
2   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\DONGES - MAIRIE - Réno...  250376           DONGES     MAIRIE            Rénov périscolaire et extrascolaire  Chantiers
6   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\MONTAGNE - MAIRIE - Ce...  230873         MONTAGNE     MAIRIE  Centre technique municipal(En attente budget)  Chantiers
7   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\PORNIC - MAIRIE - Remp...  250425           PORNIC     MAIRIE                        Remplacement couverture  Chantiers
10  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\PORNICHET - MAIRIE - G...  241142        PORNICHET     MAIRIE   

In [8]:
df_consult.head()

,SPS Name,Directory,ID EBP,Ville,Entreprise,Mission,statut,filename
0,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE B...,,BERNERIE,LAD,PRE BOISMAIN,Chantiers,CCTP_CSPS_Tranche 2 définitive.pdf
1,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS CCAP.pdf
2,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS CCTP.pdf
3,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS RC.pdf
4,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\03-EN ATTENTE PHASE 2\CHAUMES EN ...,,CHAUMES EN RETZ,PORNIC AGGLO,Tx Alvéoles ISDND Écocentre Ste Anne,Chantiers,2023_300_CCTP_SPS_Travaux_alveoles_ISDND_Ecoce...


In [9]:
filtered_consult = df_consult[
    (df_consult['SPS Name'] == "Cyrille CHARTIER - CYRILLE") &
    (df_consult['statut'] == "Chantiers") &
    (df_consult['Entreprise'] == "MAIRIE")  
].head(10)

print(filtered_consult)

                      SPS Name                                          Directory ID EBP           Ville Entreprise                       Mission     statut               filename
1   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...         MONTOIR DE BZH     MAIRIE  Ecole Jaurès & multi accueil  Chantiers  EJJ-MPE CSPS CCAP.pdf
2   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...         MONTOIR DE BZH     MAIRIE  Ecole Jaurès & multi accueil  Chantiers  EJJ-MPE CSPS CCTP.pdf
3   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...         MONTOIR DE BZH     MAIRIE  Ecole Jaurès & multi accueil  Chantiers    EJJ-MPE CSPS RC.pdf
14  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\04-EN COURS\PORNICHET - MAIRIE - ...              PORNICHET     MAIRIE              REHAB VIEUX MOLE  Chantiers               CCAP.pdf
15  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\04-EN COURS\PORNICHET - MAIRIE - ...              PORNI

In [10]:
df_consult = df_consult[~df_consult['filename'].str.contains("Attestation", na=False)].copy()

In [11]:
TableReport(df_consult)

Processing column   8 / 8


,,,,,,,,


In [12]:
df_consult['statut'].value_counts()

statut
Archive                       914
Chantiers                     601
Annule                        346
Offre en cours de decision     35
Perdu                          18
unknown                        15
Devis en cours                  3
Name: count, dtype: int64

In [13]:
df_EBP['statut'].value_counts()

statut
Chantiers                     1742
Archive                       1206
Annule                          88
Offre en cours de decision       3
unknown                          3
Perdu                            1
Name: count, dtype: int64

In [14]:
print(df_consult[df_consult['filename'].str.contains("Attestation", na=False)])
print(len(df_consult['filename']))
df_consult = df_consult[~df_consult['filename'].str.contains("Attestation", na=False)].copy()
print(len(df_consult['filename']))

print(df_consult[df_consult['filename'].str.contains("engagement", na=False)])


Empty DataFrame
Columns: [SPS Name, Directory, ID EBP, Ville, Entreprise, Mission, statut, filename]
Index: []
1932
1932
                         SPS Name                                          Directory ID EBP                         Ville                                         Entreprise                        Mission     statut                         filename
1063  Feliciano AMARO - FELICIANO  01-CHANTIERS\00-NOUVEAU\RENNES - TERRITOIRES P...                               RENNES                                TERRITOIRES PUBLICS  TP 103 ATELIERS DE MAUREPAS 2  Chantiers  Acte_engagement_valant_CCAP.pdf
1071  Feliciano AMARO - FELICIANO  01-CHANTIERS\03-EN ATTENTE PHASE 2\RENNES - TE...         RENNES - TERRITOIRES PUBLICS  AMENAGEMENT SECTEUR DES MARGINELLES (phase 2 2025                      2026)\ADM  Chantiers  Acte_engagement_valant_CCAP.pdf
1110  Feliciano AMARO - FELICIANO  01-CHANTIERS\05-DIUO A FAIRE\A FAIRE\RENNES - ...          RENNES - TERRITOIRES RENNES              

In [15]:
# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

# Perform an inner merge on the specified columns
# An inner merge will only keep rows where the values in 'match_columns' are identical in both DataFrames.
matched_df = pd.merge(df_EBP, df_consult, on=match_columns, how='inner', suffixes=('_ebp', '_consult'))

print("there are ",len(matched_df['ID EBP_ebp'].unique()), " EBP ID matching")
print(f"Taille du fichier consolidé (ID EBP/Consult):{len(matched_df)}, Taille du fichier EBP:{len(df_EBP)}, Taille du fichier consult:{len(df_consult)}")
print(f"Nb unique EBP ID dans fichier consolidé:{len(matched_df['ID EBP_ebp'].unique())}, Nb unique EBP ID dans le fichier EBP:{len(df_EBP['ID EBP'].unique())} ")

print(matched_df.head())

there are  412  EBP ID matching
Taille du fichier consolidé (ID EBP/Consult):1431, Taille du fichier EBP:3043, Taille du fichier consult:1932
Nb unique EBP ID dans fichier consolidé:412, Nb unique EBP ID dans le fichier EBP:2591 
                     SPS Name                                      Directory_ebp ID EBP_ebp            Ville    Entreprise                               Mission statut_ebp                                  Directory_consult ID EBP_consult statut_consult                                           filename
0  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE B...     250286         BERNERIE           LAD                          PRE BOISMAIN  Chantiers  01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE B...                     Chantiers                 CCTP_CSPS_Tranche 2 définitive.pdf
1  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...     220935   MONTOIR DE BZH        MAIRIE          Ecole Jaurès & multi accueil  

In [16]:
print("nb de lignes dont le statut est different : ", len(matched_df[matched_df['statut_consult'] != matched_df['statut_ebp']]))
print("nb de lignes dont le directory est different : ", len(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]))
print(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]['Directory_consult'][0])
print(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]['Directory_ebp'][0])

nb de lignes dont le statut est different :  2
nb de lignes dont le directory est different :  1431
01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE BOISMAIN\ADMINISTRATIF\Consult\
01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE BOISMAIN\


In [17]:
matched_df =  matched_df.drop(columns=['ID EBP_consult','Directory_ebp'])

In [18]:
TableReport(matched_df)

Processing column   9 / 9


,,,,,,,,,


# Test de modele pour resumer et extraire l'info un CCTP 

In [23]:
# test load file

num_cctp_test = 0

path_root = r"C:\Users\jch_m\ATAE"
print(matched_df['SPS Name'][num_cctp_test])
print(matched_df['Directory_consult'][num_cctp_test])
print(matched_df['filename'][num_cctp_test])

file_path = os.path.join(path_root, matched_df['SPS Name'][num_cctp_test], matched_df['Directory_consult'][num_cctp_test], matched_df['filename'][num_cctp_test])
print(file_path)


Cyrille CHARTIER - CYRILLE
01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE BOISMAIN\ADMINISTRATIF\Consult\
CCTP_CSPS_Tranche 2 définitive.pdf
C:\Users\jch_m\ATAE\Cyrille CHARTIER - CYRILLE\01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE BOISMAIN\ADMINISTRATIF\Consult\CCTP_CSPS_Tranche 2 définitive.pdf


In [24]:
from pdfquery import PDFQuery
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

pdf = PDFQuery(file_path,)
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements if t.text != '' or t.text != ' ']

#print(text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [25]:
from ollama import Client

ollama_url = 'http://localhost:11434'
#model_name = 'llama3.1:8b'
model_name = 'gemma3:4b'

# Get an ollama client
llmclient = Client(host=ollama_url)

model_options = {
    'num_predict': 1300,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}

#prompt_task = "summarize in french this text and highlight key points of the mission and bid features to consider in less than 900 words\n"
prompt_task = "En Francais, résume ce texte et extrait les points importants de la mission et les caractéristiques du contrat (prix, durée, lieu, nom du maitre d'oeuvre, nom du lieu)en moins de 900 mots\n"

prompt_full = prompt_task + "".join(text)
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

GenerateResponse(model='gemma3:4b', created_at='2025-06-06T07:18:15.3331563Z', done=True, done_reason='stop', total_duration=163489472400, load_duration=5296503500, prompt_eval_count=3603, prompt_eval_duration=79131378500, eval_count=854, eval_duration=79059734300, response="Voici un résumé du texte, extrait des points importants de la mission et des caractéristiques du contrat, en moins de 900 mots :\n\n**Résumé Général**\n\nCe document est le Cahier des Clauses Techniques et Particulières (CCTP) pour un marché de prestations intellectuelles concernant la coordination sécurité et protection de la santé (SPS) dans le cadre des travaux d'aménagement de la ZAC du Pré-Boismain à la Bernerie-en-Retz, Loire-Atlantique. Le marché est géré par Loire-Atlantique Développement (LAD-SELA) en tant que concessionnaire-aménageur pour le compte de Pornic Agglomération Pays de Retz. L’objectif est de garantir la sécurité et la protection de la santé des travailleurs sur le chantier.\n\n**Points Import

In [ ]:
"""
model_name = 'deepseek-r1:7b'

model_options = {
    'num_predict': 1500,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])
"""

In [26]:
model_name = 'llama3.2:latest'

model_options = {
    'num_predict': 1300,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

GenerateResponse(model='llama3.2:latest', created_at='2025-06-06T07:20:28.7114069Z', done=True, done_reason='stop', total_duration=133298964700, load_duration=3628772800, prompt_eval_count=4096, prompt_eval_duration=93010248600, eval_count=413, eval_duration=36657781700, response="Voici la version complète du document :\n\n**CCTP SPS - ZAC du Pré-Bois**\n\n**Phase de réalisation des travaux**\n\nLe présent marché a pour objet la réalisation des travaux suivants :\n\n*   Travaux de VRD (hors travaux des concessionnaires / hors éclairage public) : environ 95 000 € HT\n*   Travaux d'espaces verts : environ 95 000 € HT\n\n**Mission de conseil et d'accompagnement**\n\nLa mission de conseil et d'accompagnement au maître d'ouvrage sera réalisée par la participation et l'animation d'un ensemble de réunions.\n\n*   Phase conception : 2 mois\n*   Phase réalisation : 4 mois\n\n**Prix unitaires**\n\nLe prix unitaire pour cette mission sera forfaitaire, la composition du prix sera détaillée selon l

In [ ]:
TableReport(df_rejet)

# Etude des Rejets

In [28]:
print(df_rejet['SPS Name'][20])
print(df_rejet['Directory'][20])
print(df_rejet['filename'][20])

file_path = os.path.join(path_root, df_rejet['SPS Name'][20], df_rejet['Directory'][20], df_rejet['filename'][20])
print(file_path)
print(os.path.exists(file_path))

Eric Garnier - ERIC
00-DEVIS\2022\01-ECHEC 2022\VERTOU - MAIRIE - Construction GS Echalonnieres\Notification refus\
lr-atae-achsig.pdf
C:\Users\jch_m\ATAE\Eric Garnier - ERIC\00-DEVIS\2022\01-ECHEC 2022\VERTOU - MAIRIE - Construction GS Echalonnieres\Notification refus\lr-atae-achsig.pdf
True


In [27]:
list_fichier_rejet_valid = []

for i in range(len(df_rejet)):
    file_path = os.path.join(path_root, df_rejet['SPS Name'][i], df_rejet['Directory'][i], df_rejet['filename'][i])
    if os.path.exists(file_path):
        try: # certains nom de fichiers provoquent des erreurs
            pdf = PDFQuery(file_path)
            pdf.load()

            # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

            # Use CSS-like selectors to locate the elements
            text_elements = pdf.pq('LTTextLineHorizontal')
            if len(text_elements) > 20:
                #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

                # Extract the text from the elements
                text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
                list_fichier_rejet_valid.append(i)
            else:
                print(i, " === pas de text ", file_path )
                pass
        except Exception as e:
            print(f"Error processing {file_path}: {e}")        
    else:
        print(i, " === Existe pas ", file_path )
        pass


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


2  === pas de text  C:\Users\jch_m\ATAE\Cyrille CHARTIER - CYRILLE\00-DEVIS\2024\00-Echec 2024\ST NAZAIRE - SONADEV - CONSTRUCTION GROUPE SCOLAIRE\REFUS\atae.pdf
3  === pas de text  C:\Users\jch_m\ATAE\Cyrille CHARTIER - CYRILLE\00-DEVIS\2025\00-ECHEC 2025\ST JEAN DE BOISEAU -IME SESSAD  - Construction établissement\Notification\ime.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


8  === pas de text  C:\Users\jch_m\ATAE\Eric Garnier - ERIC\00-DEVIS\2021\0-ECHEC 2021\MONTAIGU - TERRES DE MONTAIGU - Finition aménagement quartier de la Gare\Notification refus.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

170  === pas de text  C:\Users\jch_m\ATAE\Yann HERVE - YANN\00-DEVIS\DEVIS - ARCHIVES\Devis - ECHEC 2022\CARQUEFOU - MAIRIE - MS13 Parking Moulin Boisseau (résiliation marché)\ADMINISTRATIF\NOTIFICATION DU MS13\20S0037 - DPGF - ATAE - MS13.pdf


CropBox missing from /Page, defaulting to MediaBox


171  === pas de text  C:\Users\jch_m\ATAE\Yann HERVE - YANN\00-DEVIS\DEVIS - ARCHIVES\Devis - ECHEC 2022\CARQUEFOU - MAIRIE - MS13 Parking Moulin Boisseau (résiliation marché)\ADMINISTRATIF\NOTIFICATION DU MS13\20S0037 - Notification ATAE - MS13.pdf
172  === pas de text  C:\Users\jch_m\ATAE\Yann HERVE - YANN\00-DEVIS\Devis Echec 2024\MONTAIGU - CC TERRES DE MONTAIGU-  RUE DE LA GARE\regrets ATAE.pdf


CropBox missing from /Page, defaulting to MediaBox


173  === pas de text  C:\Users\jch_m\ATAE\Yann HERVE - YANN\00-DEVIS\Devis Echec 2025\BRUFFIERE - MAIRIE TERRES DE MONTAIGU - Terrain synthétique\Notif refus\Notif refus BRUFFIERE Terrain foot.pdf


In [30]:
print("Nb de fichiers de rejet valides :",len(list_fichier_rejet_valid), " sur ", len(df_rejet), "fichiers")
print(list_fichier_rejet_valid)

#file_path = os.path.join(path_root, df_rejet['SPS Name'][104], df_rejet['Directory'][104], df_rejet['filename'][104])
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
print(file_path)

Nb de fichiers de rejet valides : 72  sur  174 fichiers
[0, 1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 17, 18, 19, 26, 27, 29, 30, 31, 35, 37, 39, 40, 42, 44, 45, 46, 49, 51, 52, 53, 55, 60, 61, 62, 66, 68, 70, 71, 72, 78, 80, 81, 84, 87, 88, 92, 93, 94, 96, 99, 104, 105, 106, 112, 116, 117, 122, 125, 133, 135, 139, 140, 141, 142, 144, 151, 155, 158, 159, 160, 163]
C:\Users\jch_m\ATAE\Fabien ROUILLE - FABIEN\00-DEVIS\ARCHIVE DEVIS\DEVIS 2023\00-ECHEC 2023\SGXV - DEPT85 - Collège Pierre Garcie Ferrande Extension\REFUS\CourrierRejetOffre-8160-ATAE-2023CO-M0042-.pdf


In [ ]:
# Creer une liste des documents de rejet "valide" (fichier exist et contient du text)
#
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
if os.path.exists(file_path):
    print("Fichier choisi :",file_path)
    pdf = PDFQuery(file_path)
    pdf.load()
    # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    if len(text_elements) > 20:
        #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

        # Extract the text from the elements
        text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
    else:
        print("ERREUR : pas de text dans le pdf")
else:
    print("ERREUR: le fichier n'est pas accessible ")


CropBox missing from /Page, defaulting to MediaBox


Fichier choisi : C:\Users\jch_m\ATAE\Fabien ROUILLE - FABIEN\00-DEVIS\ARCHIVE DEVIS\DEVIS 2022\01-ECHEC 2022\SABLES - MAIRIE - Bât annexe de la Jarrie\Notification refus\Courrier non retenu CSPS - ATAE.pdf


In [ ]:
# Execution du modele pour 3 exemples de fichiers "rejet" ayant du text_elements
model_name = 'llama3.2:latest'

model_options = {
    'num_predict': 1300,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}

prompt_task = "summarize this text in french and highlight the reason for the ATAE bid failure, the motivation for the decision , the bid winner name and the price awarded in less than 500 words\n"

# test 3 fichiers parmi les valides
for i in range(3):
    print("\n====== Test ", i+1)
    item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
    file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
    if os.path.exists(file_path):
        print("Fichier choisi :", item_valid_rejet_file," : ",file_path)
        pdf = PDFQuery(file_path)
        pdf.load()
        # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

        # Use CSS-like selectors to locate the elements
        text_elements = pdf.pq('LTTextLineHorizontal')
        if len(text_elements) > 20:
            #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

            # Extract the text from the elements
            text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
        else:
            print("ERREUR : pas de text dans le pdf")
            break
    else:
        print("ERREUR: le fichier n'est pas accessible ")
        break
        
    prompt_full = prompt_task + "".join(text)

    result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

    pprint(result, compact=False, sort_dicts=False)

    print("LLM response:\n", result['response'])

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



====== Test  1
Fichier choisi : 55  :  C:\Users\jch_m\ATAE\Fabien ROUILLE - FABIEN\00-DEVIS\ARCHIVE DEVIS\DEVIS 2023\00-ECHEC 2023\SGXV - DEPT85 - Collège Pierre Garcie Ferrande Extension\REFUS\CourrierRejetOffre-8160-ATAE-2023CO-M0042-.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


GenerateResponse(model='llama3.2:latest', created_at='2025-06-06T07:33:26.9524479Z', done=True, done_reason='stop', total_duration=47808106500, load_duration=36900700, prompt_eval_count=816, prompt_eval_duration=12860618500, eval_count=507, eval_duration=34910068200, response="Voici un résumé du texte en français :\n\nLa Direction Juridique du Conseil Départemental de Vendée a émis une décision relative à l'attribution d'un contrat pour la réalisation de missions de contrôle technique et de coordination sécurité et protection de la santé dans le cadre de l'extension capacitaire du collège Pierre Garcie-Ferrande de Saint-Gilles-Croix-de-Vie.\n\n**Raisonnement de la décision :**\n\nLa décision a été prise en raison d'une évaluation négative des candidatures, notamment pour les raisons suivantes :\n\n* La qualité des interlocuteurs mobilisés pour l'opération (12 pts) : 9,00/12,00\n* La cohérence du temps d'intervention, tant en phase de conception que de réalisation au regard de la comple

CropBox missing from /Page, defaulting to MediaBox


GenerateResponse(model='llama3.2:latest', created_at='2025-06-06T07:34:05.0930442Z', done=True, done_reason='stop', total_duration=37787230300, load_duration=32874300, prompt_eval_count=701, prompt_eval_duration=11106053900, eval_count=393, eval_duration=26647796500, response="Voici un résumé du texte en français :\n\nLa commune de Châumes-en-Retz a publié un appel d'offres (AO) pour la réalisation de missions annexes dans le cadre des travaux d'extension et de transformation d'une maison d'habitation en accueil périscolaire à La Sicaudais. L'AO était divisé en deux lots, dont le Lot 2 concernant la CSPS.\n\nLa société ATAE a soumis une proposition pour le Lot 2, mais elle n'a pas été retenue. En effet, son offre arrive en troisième position sur trois propositions reçues.\n\n**Raison du échec de l'AO :**\n\n* La société ATAE n'a pas atteint les critères de présence en phase conception et en phase exécution requis par le règlement de consultation.\n* Son offre a été jugée inférieure à c

## Consolidation rejet

In [37]:
df_EBP.head(10)

,SPS Name,Directory,ID EBP,Ville,Entreprise,Mission,statut
0,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE B...,250286,BERNERIE,LAD,PRE BOISMAIN,Chantiers
1,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\CHAUMES EN RETZ - MAIR...,250492,CHAUMES EN RETZ,MAIRIE,ancien bureau de poste,Chantiers
2,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\DONGES - MAIRIE - Réno...,250376,DONGES,MAIRIE,Rénov périscolaire et extrascolaire,Chantiers
3,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\LA MONTAGNE - DPT 44 -...,250475,LA MONTAGNE,DPT 44,Collège St Exupéry Accès,Chantiers
4,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\LA MONTAGNE - DPT44 - ...,250470,LA MONTAGNE,DPT44,Coll St Exupéry Terrasses,Chantiers
5,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\LA MONTAGNE - DPT44 - ...,250472,LA MONTAGNE,DPT44,Coll St Exupéry Terrasses,Chantiers
6,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\MONTAGNE - MAIRIE - Ce...,230873,MONTAGNE,MAIRIE,Centre technique municipal(En attente budget),Chantiers
7,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\PORNIC - MAIRIE - Remp...,250425,PORNIC,MAIRIE,Remplacement couverture,Chantiers
8,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\PORNICHET - CISN - Rés...,240939,PORNICHET,CISN,Résidences Habitat Jeunes-Séniors,Chantiers
9,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\PORNICHET - LEXHAM - E...,240951,PORNICHET,LEXHAM,Espace Santé,Chantiers


In [38]:
df_rejet.head(10)

,SPS Name,Directory,ID EBP,Ville,Entreprise,Mission,statut,filename
0,Cyrille CHARTIER - CYRILLE,00-DEVIS\2022\00-Echec 2022\BOUGUENAIS - BGTA ...,,BOUGUENAIS,BGTA Ministère environnement,Sécurisation BGTA\REFUS,Perdu,SNIA-Ouest-22-006_notif_offre_non_retenue_ATAE...
1,Cyrille CHARTIER - CYRILLE,00-DEVIS\2023\00-Echec 2023\CHAUMES EN RETZ - ...,,CHAUMES EN RETZ,PORNIC AGGLO,APS La Sicaudais\REFUS,Perdu,2023_23_L2_regret_ATAE.pdf
2,Cyrille CHARTIER - CYRILLE,00-DEVIS\2024\00-Echec 2024\ST NAZAIRE - SONAD...,,ST NAZAIRE,SONADEV,CONSTRUCTION GROUPE SCOLAIRE\REFUS,Perdu,atae.pdf
3,Cyrille CHARTIER - CYRILLE,00-DEVIS\2025\00-ECHEC 2025\ST JEAN DE BOISEAU...,,ST JEAN DE BOISEAU,IME SESSAD,Construction établissement\Notification,Perdu,ime.pdf
4,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\BARRE DE MONTS - OR...,,BARRE DE MONTS,ORYON,8 Logements Querruy\Notification refus,Perdu,Rapport d'analyse SPS LBDM (1).pdf
5,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\CHEVROLIERE - ACIST...,,CHEVROLIERE,ACISTE ING,Restauration église\Notification refus,Perdu,SPS_ATAE.pdf
6,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\HERBERGEMENT - PODE...,,HERBERGEMENT,PODELIHA,Le Clos des Bambous\Notification refus,Perdu,139 09 2021 ATAE.pdf
7,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\MONTAIGU - CC TERRE...,,MONTAIGU,CC TERRES DE MONTAIGU,Tx aménagemt voirie & d’assainissemt EU et EP ...,Perdu,refus offre ATAE.pdf
8,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\MONTAIGU - TERRES D...,,MONTAIGU,TERRES DE MONTAIGU,Finition aménagement quartier de la Gare,Perdu,Notification refus.pdf
9,Eric Garnier - ERIC,00-DEVIS\2021\0-ECHEC 2021\NANTES - REGION PDL...,,NANTES,REGION PDL,Lycée François Arago\REJET,Perdu,COURRIERS_REGRET_ARAGO_CSPS-ATAE_1.pdf


In [39]:
# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

# Perform an inner merge on the specified columns
# An inner merge will only keep rows where the values in 'match_columns' are identical in both DataFrames.
reject_c_df = pd.merge(df_rejet, matched_df, on=match_columns, how='inner', suffixes=('_ebp', '_consult'))

print("there are ",len(reject_c_df), " rejet matching")
print(f"Taille du fichier consolidé (consult/rejet):{len(reject_c_df)}, Taille du fichier consult:{len(matched_df)}, Taille du fichier rejet:{len(df_rejet)}")
#print(f"Nb unique EBP ID dans fichier consolidé:{len(reject_c_df['ID EBP_ebp'].unique())}, Nb unique EBP ID dans le fichier EBP:{len(df_EBP['ID EBP'].unique())} ")

print(reject_c_df.head())

there are  0  rejet matching
Taille du fichier consolidé (consult/rejet):0, Taille du fichier consult:1431, Taille du fichier rejet:174
Empty DataFrame
Columns: [SPS Name, Directory, ID EBP, Ville, Entreprise, Mission, statut, filename_ebp, ID EBP_ebp, statut_ebp, Directory_consult, statut_consult, filename_consult]
Index: []
